 정수 n 이 주어졌을 때 n 번째 피보나치 수열의 항을 계산하는 함수를 작성하시오.
 
 예)
 $fib(0) = 0$ 

 $fib(1) = 1$
 
 $fib(5) = 5$ 
 
 $fib(10) = 55$
 
 ## 첫 번째 해법 찾기
 
 FAST 방법의 첫 번째 단계는 가능한 첫 번째 해법을 찾는 것이다. 이런 종류의 문제에서는 피보나치 수열의 정의에 집중한다면 그 해법을 찾을 수 있다. 만약 정의가 확실치 않다면 인터뷰어에게 물어보면 된다. 하지만, 대부분의 사람들이 학교에서 다음의 개념을 배웠을 것이다.

$fib(n) = fib(n-1) + fib(n-2)$

